## The data figure we want to reproduce (from [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3039263/figure/fig01/))

![data-figure](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3039263/bin/tjp0589-0101-f1.jpg)

In [1]:
%matplotlib inline

# Built-in libs
import os
import sys
import warnings

# Common libs
import matplotlib.pyplot as plt
#!pip install quantities
import quantities as pq

# Open Worm libs
import owtests
import c302

# Testing libs
warnings.simplefilter('ignore')
import sciunit
from neuronunit.tests import APWidthTest,APAmplitudeTest
from c302.muscleunit import vm_plot,models as sciunit_models

FileNotFoundError: No directory found at ../CElegansNeuroML.  Please set OW_HOME to the location of your OpenWorm projects directory.

In [ ]:
# Create a MuscleModel instance with a particular configuration
# This model has a neuron and a muscle cell
model = sciunit_models.MuscleModel('IClampBWM', 'C2', 1000, 0.05, config_package="notebooks.configs")

In [ ]:
# Dictionary of observations, in this case two ephys properties from one paper
from neuronunit.tests import dynamics 
import quantities as pq

doi = 'doi:10.1113/jphysiol.2010.200683'
observations={doi:{'ap_amplitude':{'mean':45.1*pq.mV,
                                   'sem':0.7*pq.mV,
                                   'n':25},
                   'ap_width':{'mean':19.7*pq.ms,
                               'sem':1.0*pq.ms,
                               'n':25}}}    

# Instantiate two tests based on these properties
ap_width_test = APWidthTest(observation=observations[doi]['ap_width'])
ap_amplitude_test = APAmplitudeTest(observation=observations[doi]['ap_amplitude'])



observation = {}
observation[doi] = {}
observation[doi]['isi'] = 598.0*pq.ms
observation[doi]['mean'] = 598.0*pq.ms
observation[doi]['std'] = 37.0*pq.ms
isi_test = dynamics.ISITest(observation=observation[doi])
# Put them together in a test suite


observation = {}
observation[doi] = {}
observation[doi]['isi'] = 16.1
observation[doi]['mean'] = 16.1*pq.ms
observation[doi]['std'] = 2.1*pq.ms
isi_test = dynamics.FiringRateTest(observation=observation)

ap_tests = sciunit.TestSuite([ap_width_test,ap_amplitude_test,isi_test], name="AP and ISI Tests")

In [ ]:
# Judge the membrane potential of the muscle 
# One could judge the neuron instead with appropriate keyword parameters, see below)
%time score_matrix = ap_tests.judge(model) 
import pandas as pd
score_matrix.style.set_properties(**{'font-size':'16pt'})
score_matrix

In [ ]:
# Plot the model output for one of the tests
import matplotlib as mpl
mpl.rcParams.update({'font.size':18})
plt.figure(figsize=(12,6))
score_matrix['APWidthTest']['LEMS_c302_C2_IClampBWM'].plot_vm()
plt.savefig('model.pdf')

## Model methods could also be run directly as follows

In [ ]:
# These will use the cached results and so be much faster than the original run
%time neuron_vm = model.get_membrane_potential_neuron()
%time muscle_vm = model.get_membrane_potential_muscle()

In [ ]:
# Another way to plot the model output, in this case by cell type
vm_plot(muscle_vm,"Muscle")

In [ ]:
# Same as above, but for a different cell type in this model
# Note that in this flavor of the model, the neuron isn't doing anything at all, 
# and the muscle cell is firing spontaneously
vm_plot(neuron_vm,"Neuron")

In [ ]:
from IPython.display import Image 
Image(filename='worm_muscle.png')

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame()
df = df.from_csv('digitized_histogram.csv')
columns = df.columns[0]
histogram_values = list(df.values)
histogram_values = [ float(i) for i in histogram_values ]
histogram_values.insert(0,float(df.columns[0]))
print(len(histogram_values))
cv_axis = np.linspace(0,2,len(histogram_values))
print(len(cv_axis))

df2 = pd.DataFrame(list(zip(cv_axis,histogram_values)))
from matplotlib.pyplot import bar
from matplotlib import pyplot as plt

fig = plt.figure()

width =  cv_axis[1] - cv_axis[0]
ind = cv_axis
plt.bar(cv_axis, histogram_values, width=width)
plt.title('Coefficient of Variation')
plt.show()
print(mean_CV)